# Final Assignment: A morphogical analyzer

In the final assignment, you will build and evaluate an algorithm that learns to segment words into their morphemes. That is to say: the algorithm should be able to determine that _unbelievers_ breaks down into _un-_, _believ_, _-er_ and *-s*.

## The Algorithm

The algorithm we will use for doing so is an unsupervised learner. An unsupervised learner is a learner that is not given training data with the correct labels or segmentations but works by just analyzing the unlabeled/unsegmented data it receives. The intuition of the model we will work with is the following: if you have word pairs like _unbeliever_ and _unbelievers_, and _book_ and _books_, and _table_ and _tables_, and many more singular--plural pairs, you can (without knowing the language) make the educated guess that _-s_ is a bound morpheme (an affix) in the language.

The model starts with a set of Unmodeled words, which is initialized to be the entire vocabulary (all unique words) for a language, as well as a empty Derivation dictionary, which will contain the analyses. The model then (1) finds the pair of affixes (including zero-affixes) that models the highest number of word pairs among the Unmodeled words, and (2) creates entries in a Derivation dictionary with each member of each word pair as the key and its induced stem + affix derivation as the value. In step (3), finally, the words in the word pair are removed from the Unmodeled set, and the resulting stem is added to the Unmodeled set. Let's have a look at an example to see how that all works!

## An example

Let's walk through an example of the algorithm. Suppose we have a set of unmodeled words:

`unmodeled = { 'book', 'bookers', 'books', 'booker', 'chair', 'chairs', 'table', 'tables', 'tags' }`

### Getting a list of most likely affixes

We start by extracting all possible likely affixes. Let's limit them to be **maximally length 2**. Doing so we find the following suffixes (make sure you understand why they are associated with the numbers -- can you trace the words they come from?):

* '' (9x), '-k' (1x), '-ok' (1x), '-s' (5x), '-rs' (2x), '-ks' (1x), '-r' (2x), '-er' (1x), '-ir' (1x), '-le' (1x), '-es' (1x), '-gs' (1x)

and the following prefixes:

* 'b-' (4x), 'bo-' (4x), 'c-' (2x), 'ch-' (2x), 't-' (3x), 'ta-' (3x)

(note that we only count the empty string as a suffix -- we don't _also_ count it as a prefix)

Since comparing all affixes to each other to see how many stems they model together would take too long when using larger vocabularies (the number of comparisons would get too large), we start by extracting a top-$k$ list of affixes, ranked on their **affix score**. We calculate the affix score by multiplying the count of that affix in the vocabulary (on how many words does it occur) by the character length of the affix plus 1. The equation below summarizes the affix score $score$ of an affix $a$:

# $score(a) = freq(a) * (len(a)+1)$

The length modifier is part of the affix score, as longer substrings (like _-ness_) are often less common by themselves, while still being good affixes, versus shorter substrings (like _b-_ in the example above) that may be frequent but that are not good affixes. (Why the +1, you may ask. If we don't add 1, zero-length strings will have a score of 0 no matter their frequency, which is what we want to avoid.)

So, if we rank our affixes by their affix score, we get:

* 'bo-' (12), '-s' (10), 'ta-' (9), '' (9), 'b-' (8), , 'ch-' (6), '-rs' (6), 't-' (6), '-r' (4), 'c-' (4), '-ok' (3),'-ks' (3), '-er' (3), '-ir' (3), '-le' (3), '-es' (3), '-gs' (3), '-k' (2)

From this ranked list, we take the list of **most likely affixes**, or the top-$k$ affixes. Here, we only have 19, so there is no great need to cut the list, but if you have thousands of them, you will have to limit the next steps to the top $k$ ranked ones. Let's take the top-18 here just to illustrate the principle:

* 'bo-' (12), '-s' (10), 'ta-' (9), '' (9), 'b-' (8), , 'ch-' (6), '-rs' (6), 't-' (6), '-r' (4), 'c-' (4), '-ok' (3), '-rs' (3), '-ks' (3), '-er' (3), '-ir' (3), '-le' (3), '-es' (3)

### Getting the stems per affix

If we want to find out how many stems overlap for an affix pair, a good first step is to gather the stems per affix. To do so, we will create a dictionary where every affix in the most likely affix list is mapped onto all the stems it models. The stems are the remaining part of the word string after we have taken the affix away. So, for the first four affixes in our list, we get:

`{ ('prefix', 'bo') : { 'ok', 'oks', 'oker', 'okers' }
   ('suffix', 's')  : { 'book', 'booker', 'chair', 'table', 'tag' }
   ('prefix', 'ta') : { 'ble', 'bles', 'gs' }
   ('suffix', '')   : { 'book', 'books', 'booker', 'bookers', 'chair', 'chairs', 'table', 'tables', 'tags' } 
   ...
}`

### Select the most likely affix pair

Now that we have the sets of stems per affix, we can compare pairs of affixes and determine how many overlapping stems they have. From among the four above, we find the following comparisons:

* 'bo-', '-s' have 0 overlapping stems
* 'bo-', 'ta-' have 0 overlapping stems
* 'bo-', '' have 0 overlapping stems
* '-s', 'ta-' have 0 overlapping stems
* '-s', '' have 4 overlapping stems: { 'book', 'booker', 'chair', 'table' }

Other affix pairs with overlapping stems are:

* '-s', '-er' have 1 overlapping stem: { 'book' }
* '', '-er' have 1 overlapping stem: { 'book' }

We take the **most likely affix pair** to be that pair that has __the most shared stems__. In this case, that's the pair of `('suffix', '')` and `('suffix', 's')`.

### Adding the derivation

We determined that the pair of `('suffix', '')` and `('suffix', 's')` is the most likely affix in the language. What we do next, is add the analysis of each of the stems that this pair models to the dictionary of derived forms. That is: we add a key that is the full word (stem combined with affix) and assign to that key the value which is a pair of the stem and the affix. So, whereas `derived` was empty before, now it contains:

`{ 'book'    : ('book', ('suffix', '')),
  'books'   : ('book', ('suffix', 's')),
  'booker'  : ('booker', ('suffix', '')),
  'bookers' : ('booker', ('suffix', 's')), 
  'chair'   : ('chair', ('suffix', '')),
  'chairs'  : ('chair', ('suffix', 's')), 
  'table'   : ('table', ('suffix', '')),
  'tables'  : ('table', ('suffix', 's')) }`

This dictionary will allow us to later look up a word's analysis.

### Replacing the modeled words in unmodeled

Finally, we remove the modeled words from the set of Unmodeled words (after all, they are no longer unmodeled), and add the stems (which may still need to be analyzed further).

Stepwise, this works as follows. For the affix pair `('suffix', '')` and `('suffix', 's')`, we find 8 words modeled by the pair (the ones added to the derivation dictionary in the previous step). Removing these words from `unmodeled` yields:

`unmodeled = { 'tags' }`

Next, adding the 4 stems (which happen to be identical to some of the removed words, but that's not necessarily the case) yields:

`unmodeled = { 'book', 'booker', 'chair', 'table', 'tags' }`

### Back to the top

And once we've done this, we go back to the step where we get the most likely affix list (from the _updated_ list of Unmodeled words and again extract the most likely affix pair, add it to the derivation dictionary and update the unmodeled words. We repeat this procedure $n$ times in total, where we can set $n$ to be any value between 1 and infinity.

The next most likely affix pair for our new list of unmodeled words will be `('suffix', 'er')`, `('suffix', '')`, modeling _book_. We update the derivation dictionary so that 'booker' now is mapped onto 'book' and ('suffix', 'er'). This effectively allows us to find the full derivation for a word like _bookers_: if we give it to the derivation dictionary, we find 'booker' and ('suffix', 's') associated with it. If we then look up the "stem" in the derivation dictionary again, we find that it maps onto 'book' and ('suffix', 'er'). So, we know that _bookers_ should be analysed as 'book' + ('suffix', 'er') + ('suffix', 's').

## An overview of the Notebooks

The remaining Notebook will guide you through the following exercises:

* Building the model;
* Evaluating the model and analyzing the results;
* Applying the model to a novel language and analyzing the results.

## A note on autograding

Many parts of this Assignment have autograding functions that check if your code functions properly. In those cases, I always give a 'test time' tip so you can verify yourself as well whether it worked.

## Part 1: Building the model

Start by importing some useful libraries:

In [1]:
import nltk
nltk.data.path.append('/var/jupyterhubdata/courses/LIN340H5/data/nltk_data')
from nltk.corpus import words
from collections import Counter, defaultdict

### Getting the potential affixes per word (2pt)

Begin by creating a function that takes a word and a parameter specifying the maximum affix length as its parameters and extracts all the potential affixes that are part of the word. That is: for a word like _bookings_ and a maximum affix length of 3, you want the function to return a list of affixes as below:

`[('suffix', ''),
 ('prefix', 'b'),
 ('suffix', 's'),
 ('prefix', 'bo'),
 ('suffix', 'gs'),
 ('prefix', 'boo'),
 ('suffix', 'ngs')]`


In this Assignment, we represent affixes as pairs of they **affix type** ('prefix' or 'suffix') as a string, and the **affix string** as a string as well. So, the **suffix** _-ngs_ in _bookings_ is represented as `('suffix', 'ngs')`.

One crucial point is that we **only extract the zero-affix once**. We assume that every word always has a zero-affix, which makes it alternate with non-zero-affixes. **We assume that the zero affix is a suffix**. That is: every word always returns `('suffix', '')` as one of its affixes, but not `('prefix', '')`. (The reason is that otherwise zero-prefixes and zero-affixes would alternate with each other which would cause an infinite loop of that pair being the most likely affix pair).

#### Tips!

To get prefixes and suffixes, use slices of the word string. The `range()` operator is your friend as well!

#### Test time!

When running it on the word _bookings_ with a maximum affix length of 3, it should return the list of pairs specified above.
 
 Make sure this is the case before you continue! You have the code cell below the current one at your disposal for this.

In [2]:
def get_affixes(word, max_affix_length):
    affixes = [('suffix', '')]
    
    for i in range(1, max_affix_length + 1):
        affixes.append(('prefix', word[:i]))
        affixes.append(('suffix', word[-i:]))
    
    return affixes

In [3]:
get_affixes('bookings', 3)

[('suffix', ''),
 ('prefix', 'b'),
 ('suffix', 's'),
 ('prefix', 'bo'),
 ('suffix', 'gs'),
 ('prefix', 'boo'),
 ('suffix', 'ngs')]

In [4]:
# hidden tests happen here

### Getting the most likely affix (3pt)

Now that we can get the affixes per word, we can aggregate over all the affixes of all the words and determine which ones are most frequent. Affixes that occur in many words are more likely to be 'good' or 'quality' affixes than ones occurring only with very few words. And _longer_ affixes of a certain frequency are more likely to be real affixes than _shorter_ ones of the same frequency (since shorter strings are more likely generally).

The function below takes as its input three arguments:

* a list or set of unique words `words`
* the maximum length of an affix, `max_affix_length`, to be passed on to `get_affixes()`
* and a parameter `n_most_likely` that specifies the top $n$ of highest scoring affixes that you want to extract.

And is supposed to return a list `most_likely_affixes`, containing the $n$ highest-scoring affixes.

Obtaining the affix scores per affix is a two-step procedure: First, obtain the counts of all the potential affixes across all the words in `words`. Next, for every affix--count pair, multiply this score by the length of the suffix string plus one to get the affix score. 


#### Tips!

Use a Counter object (as imported in the first cell) for gathering the scores per affix: by iterating over the affixes for each of the words, you can increment the count of that affix by one every time you see the affix. Counter objects furthermore have this nifty `.most_common(n)` function that might come in handy when you try to extract the $n$ highest-scoring affixes.

#### Test time!

We will use the unmodeled set `{ 'book', 'bookers', 'books', 'booker', 'chair', 'chairs', 'table', 'tables', 'tags' }` as the running example for the first part of this Assignment. The code given below the function prints the 18 most likely affixes of maximum length two. This should give you this list (the order may vary a little depending on how you do it):

`[('prefix', 'bo'), ('suffix', 's'), ('suffix', ''), ('prefix', 'ta'), ('prefix', 'b'), ('prefix', 't'), ('prefix', 'ch'), ('suffix', 'rs'), ('prefix', 'c'), ('suffix', 'r'), ('suffix', 'ks'), ('suffix', 'le'), ('suffix', 'ir'), ('suffix', 'es'), ('suffix', 'er'), ('suffix', 'gs'), ('suffix', 'ok'), ('suffix', 'e')]
`

In [5]:
def get_most_likely_affixes(words, max_affix_length, n_most_likely):
    all_affixes = [get_affixes(word, max_affix_length) for word in words]
    affixes_count_list = [w for word in all_affixes for w in word]
    affixes_count = Counter(affixes_count_list)
    
    for k in affixes_count.keys():
        affixes_count[k] *= (len(k[1]) + 1)
    
    most_likely_affixes = [c[0] for c in affixes_count.most_common(n_most_likely)]
    return most_likely_affixes

In [6]:
unmodeled = { 'book', 'bookers', 'books', 'booker', 'chair', 'chairs', 'table', 'tables', 'tags' }
mla = get_most_likely_affixes(unmodeled, 2, 18)
print(mla)

[('prefix', 'bo'), ('suffix', 's'), ('suffix', ''), ('prefix', 'ta'), ('prefix', 'b'), ('prefix', 'ch'), ('prefix', 't'), ('suffix', 'rs'), ('prefix', 'c'), ('suffix', 'r'), ('suffix', 'ir'), ('suffix', 'es'), ('suffix', 'ks'), ('suffix', 'er'), ('suffix', 'ok'), ('suffix', 'le'), ('suffix', 'gs'), ('suffix', 'k')]


In [7]:
# hidden tests happen here

### Mapping affixes to stems (3pt)

In order to see how many overlapping stems two affixes share, the next, instrumental, step is to compile a dictionary of affixes mapped to the stems they model. 

The function `get_affix_stems_dictionary` takes a list of unmodeled words `unmodeled`, the list of most likely affixes, as well as the maximum affix length as its arguments. It returns a dictionary with affixes as keys, and **sets** of the **stems** they model as the values. That is, for the keys `('suffix', 'rs')` and the suffix `('suffix', 's')`, the respective sets are:

`{'booke', 'chai'}
 {'table', 'chair', 'tag', 'booker', 'book'}`
 
The general flow of the algorithm in this function is that you iterate over all words, and per word over all its possible affixes (with `get_affixes`). For each such affix, you check if it is in the set or list of most likely affixes (`affixes`) and if so, you extract the stem by removing the affixal part from the word. (E.g., if the word is _books_ and the suffix is _ks_, you want to extract _boo_ as the stem. Next, you add the stem to the set associated with the affix in the dictionary `affix_stems_dict`

This is the place where we further want to impose the constraints that stems have to be a certain **minimum length**. Stems can be short, like _go_ and _do_, but hardly ever are they shorter than two characters. So, we will only add the stem to the set associated with the affix in the `affix_stems_dict` if the length of the stem is 2 or greater.

#### Tips!

Here, the use of a `defaultdict` comes in handy. If you initialize `affix_stems_dict` to be a defaultdict that initializes an empty set when a new key is encountered, you will have an easier time adding new stems to the dictionary than if you use a regular dictionary. The second line of the code below does this for you. You can choose to use it or take it out and rewrite it with whatever you like (as long as the affix--stems dictionary you return passes the tests below).

#### Test time!

Run the code below to see if the affix-stem dictionary in `asd` indeed contains the sets listed above.

In [8]:
def get_affix_stems_dictionary(unmodeled, most_likely_affixes, max_affix_length):
    affix_stems_dict = defaultdict(lambda : set())
    
    for word in unmodeled:
        affixes_list = get_affixes(word, max_affix_length)
                
        for (tag, affix) in affixes_list:
            if (tag, affix) in most_likely_affixes:
                if tag == 'prefix':
                    stem_word = word[len(affix):]
                    if len(stem_word) >= 2:
                        affix_stems_dict[(tag, affix)].add(stem_word)
                
                if tag == 'suffix':
                    stem_word = word[:len(word) - len(affix)]
                    if len(stem_word) >= 2:
                        affix_stems_dict[(tag, affix)].add(stem_word)
                            
    return affix_stems_dict

In [9]:
asd = get_affix_stems_dictionary(unmodeled, mla, 2)
a1 = ('suffix', 'rs')
a2 = ('suffix', 's')
#
print(asd[a1])
print(asd[a2])
#
print()
print('full dictionary', asd)

{'chai', 'booke'}
{'chair', 'booker', 'book', 'tag', 'table'}

full dictionary defaultdict(<function get_affix_stems_dictionary.<locals>.<lambda> at 0x7ff2a600a268>, {('suffix', ''): {'chair', 'tables', 'books', 'bookers', 'booker', 'book', 'table', 'tags', 'chairs'}, ('prefix', 'c'): {'hair', 'hairs'}, ('suffix', 'r'): {'chai', 'booke'}, ('prefix', 'ch'): {'airs', 'air'}, ('suffix', 'ir'): {'cha'}, ('prefix', 't'): {'ables', 'able', 'ags'}, ('suffix', 's'): {'chair', 'booker', 'book', 'tag', 'table'}, ('prefix', 'ta'): {'ble', 'gs', 'bles'}, ('suffix', 'es'): {'tabl'}, ('prefix', 'b'): {'ookers', 'ook', 'ooker', 'ooks'}, ('prefix', 'bo'): {'oks', 'oker', 'ok', 'okers'}, ('suffix', 'ks'): {'boo'}, ('suffix', 'rs'): {'chai', 'booke'}, ('suffix', 'er'): {'book'}, ('suffix', 'k'): {'boo'}, ('suffix', 'ok'): {'bo'}, ('suffix', 'le'): {'tab'}, ('suffix', 'gs'): {'ta'}})


In [10]:
# hidden tests happen here

###  Get the most likely affix pair (3pt)

Now that we have a dictionary mapping affixes to the sets of stems they model in the set of unmodeled words, we can see which pairs of affixes share the most stems between them and select the most likely affix pair, i.e., the affix pair sharing the largest number of stems shared between them. To do so, you will need to iterate over every possible pair of affixes and for each pair calculate the number of stems shared between them.

Finish the code below to extract the most likely affix pair. The starting point (an empty pair, a score of 0, and a return statement) is already given. Work with those materials to find the most likely pair and return it.

#### Tips!

You calculate the number of stems shared between two affixes by taking the overlap (also called: intersection) between the two sets of stems the two affixes are associated with. Set intersection works with the ampersand operator `&`. So, if we have two sets, `a = {1,2,3}` and `b = {2,3,4,5}`, the statement `a & b` will give you `{2,3}`. Similarly for two sets of stems!


#### Test time!

Run the code below the function to make sure your function works. It should return `(('suffix', ''), ('suffix', 's'))`.

In [11]:
a = {'book', 'cat', 'dog'}

In [12]:
b = {'cat', 'book'}

In [13]:
len(a&b)

2

In [14]:
def get_most_likely_affix_pair(asd, words):
    most_likely_pair = None, None
    most_likely_pair_score = 0
    
    for k1 in asd:
        #print("k1: ", k1, asd[k1])
        for k2 in asd:
            #print("k2: ", k2, asd[k2])
            if k2 != k1:
                temp_score = len(asd[k1] & asd[k2])
                if temp_score > most_likely_pair_score:
                    most_likely_pair_score = temp_score
                    most_likely_pair = (k1, k2)
    return most_likely_pair

In [15]:
get_most_likely_affix_pair(asd, unmodeled)

(('suffix', ''), ('suffix', 's'))

In [16]:
##### hidden tests happen here

### Combining it all (3pt)

The function below, `learn_morphology` combines all the above and does the updating of the Unmodeled set and the Derived dictionary. The function first initializes a dictionary to hold derived forms and their analyses. Next, for a set number of iterations (given as an argument: `n_iterations`), the function 

* gets the most likely affixes
* gets the affix--stem dictionary
* retrieves the most likely affix pair as `a1` and `a2`
* it retrieves the stems shared between `a1` and `a2`
* it prints the two affixes, the number of shared stems, the length of the set of unmodeled words, and the length of the derivation dictionary
* it iterates over all the stems to update the `unmodeled` set and the `derived` dictionary

This last step is what you will have to write. How does the update work? The code iterates over all stems (as given). For each `stem` in `shared_stems`:

* Re-create the words the stems were extracted from. (So, if `a1` is `('prefix', 'un')` and `a2` is `('suffix', 'ly')`, and the stem is `'earth'`, the words `w1` and `w2` should be `'unearth'` and `'earthly'`). 
* Next, remove `w1` and `w2` from the set of unmodeled words, by using the `discard` method. (If you have a set `a = {1,2,3}` and you say `a.discard(3)`, `a` will now contain only `{1,2}`.)
* Next, create the dictionary entry for `w1` as the key and assign to it the value `(stem, a1)`.
* Do the same for `w2` as the key and `(stem,a2)` as the value.
* Add the stem to `unmodeled` so it can be analyzed in future iterations.

Once you've done that, the algorithm is ready for the next iteration. It will extract the most likely affixes from the remaining unmodeled words, retrieve an affix-stem dictionary, etcetera.

#### Test time!

The code cell below the function allows you to see if the derivation dictionary you get when running it on the toy example is correct. The returned dictionary should contain:

`{'table': ('table', ('suffix', '')),
 'tables': ('table', ('suffix', 's')),
 'booker': ('book', ('suffix', 'er')),
 'bookers': ('booker', ('suffix', 's')),
 'chair': ('chair', ('suffix', '')),
 'chairs': ('chair', ('suffix', 's')),
 'book': ('book', ('suffix', '')),
 'books': ('book', ('suffix', 's'))}`

In [17]:
x1 = 'test'

In [18]:
y1 = ''

In [19]:
x1 += y1

In [20]:
x1

'test'

In [21]:
x1 = y1 + x1

In [22]:
x1

'test'

In [23]:
set1 = {1, 2, 3}

In [24]:
def learn_morphology(unmodeled, max_affix_length, n_most_likely, n_iterations):
    derived = {}
    #
    for i in range(n_iterations):
        affixes = get_most_likely_affixes(unmodeled, max_affix_length, n_most_likely)
        asd = get_affix_stems_dictionary(unmodeled, affixes, max_affix_length)
        a1,a2 = get_most_likely_affix_pair(asd, unmodeled)
        shared_stems = asd[a1] & asd[a2]
        print('best pair:', a1,a2, '\t stems', len(shared_stems),
              'n unmodeled', len(unmodeled), 
              'n analyzed', len(derived))
        for stem in shared_stems:
            if a1[0] == 'prefix':
                w1 = a1[1] + stem
            if a1[0] == 'suffix':
                w1 = stem + a1[1]
            if a2[0] == 'prefix':
                w2 = a2[1] + stem
            if a2[0] == 'suffix':
                w2 = stem + a2[1]
            
            unmodeled.discard(w1)
            unmodeled.discard(w2)
            
            derived[w1] = (stem, a1)
            derived[w2] = (stem, a2)
            
            unmodeled.add(stem)
                
    return derived

In [25]:
unmodeled = { 'book', 'bookers', 'books', 'booker', 'chair', 'chairs', 'table', 'tables', 'tags' }
derived = learn_morphology(unmodeled, 2, 100, 5)
print('final derivation dict:', derived)

best pair: ('suffix', '') ('suffix', 's') 	 stems 4 n unmodeled 9 n analyzed 0
best pair: ('suffix', '') ('suffix', 'er') 	 stems 1 n unmodeled 5 n analyzed 8
best pair: None None 	 stems 0 n unmodeled 4 n analyzed 8
best pair: None None 	 stems 0 n unmodeled 4 n analyzed 8
best pair: None None 	 stems 0 n unmodeled 4 n analyzed 8
final derivation dict: {'booker': ('book', ('suffix', 'er')), 'bookers': ('booker', ('suffix', 's')), 'chair': ('chair', ('suffix', '')), 'chairs': ('chair', ('suffix', 's')), 'table': ('table', ('suffix', '')), 'tables': ('table', ('suffix', 's')), 'book': ('book', ('suffix', '')), 'books': ('book', ('suffix', 's'))}


In [26]:
# hidden tests happen here

## Part 2: Evaluating the model

Next, let's evaluate the quality of the derivations. First, let's extract a derivation dictionary over the unique words in the Brown corpus, as given in the code cell below. It will take a minute or maybe two to run! Note that we're extracting affixes up to length 5, looking at the 500 most frequent affixes, and run 20 iterations.

In [27]:
from nltk.corpus import brown
brown_words = set([w.lower() for w in brown.words()])
unmodeled = set(brown_words)
derived = learn_morphology(unmodeled, 5, 500, 20)

best pair: ('suffix', '') ('suffix', 's') 	 stems 5387 n unmodeled 49815 n analyzed 0
best pair: ('suffix', 'ed') ('suffix', 'ing') 	 stems 1947 n unmodeled 44430 n analyzed 10763
best pair: ('suffix', '') ('suffix', "'s") 	 stems 1651 n unmodeled 41494 n analyzed 14558
best pair: ('suffix', '') ('suffix', 'e') 	 stems 994 n unmodeled 39843 n analyzed 17311
best pair: ('suffix', '') ('suffix', 'ly') 	 stems 965 n unmodeled 38854 n analyzed 18564
best pair: ('suffix', '') ('suffix', 'er') 	 stems 583 n unmodeled 37889 n analyzed 20279
best pair: ('suffix', '') ('suffix', 'ed') 	 stems 493 n unmodeled 37307 n analyzed 20772
best pair: ('suffix', '') ('suffix', 'd') 	 stems 412 n unmodeled 36814 n analyzed 21474
best pair: ('suffix', '') ('suffix', 'y') 	 stems 391 n unmodeled 36406 n analyzed 22010
best pair: ('suffix', '') ('suffix', 'ing') 	 stems 310 n unmodeled 36015 n analyzed 22484
best pair: ('suffix', 'y') ('suffix', 'ies') 	 stems 308 n unmodeled 35705 n analyzed 22875
best pair

### Print derivations (2pt)

The function `parse_word` below takes a word and the derivation dictionary and prints the derivation of a word. Run it on a few words of your choice in the code cells below. Leave the words there for me to see. Try to find at least one complex (multi-morphemic) word.

In [28]:
def parse_word(word, derivation_dictionary):
    if word not in derivation_dictionary: return [word]
    else:
        stem, affix = derivation_dictionary[word]
        if stem != word:
            return [affix] + parse_word(stem, derivation_dictionary)
        else: return [affix, stem]

In [29]:
parse_word('deer', derived)

[('suffix', 'er'), ('suffix', ''), 'de']

In [30]:
parse_word('preserving', derived)

[('suffix', 'ing'), ('suffix', ''), 'preserv']

### Getting morphological families (2pt)

Morphological families are defined as sets of words that share a stem. Construct a dictionary `families` that maps each unique stem to all the words in `brown_words`.

#### Test time!

In the code cell below the following cell, two entries are printed. They should print the following two sets:

`{'books', 'booked', 'bookings', 'booker', "book's", 'bookers', 'booking', 'book'}` (for the stem 'book')

and

`{'thwarting', 'thwart', 'thwarted'}` (for the stem 'thwart')




In [31]:
families = defaultdict(lambda : set())
for word in brown_words:
    stem = parse_word(word, derived)[-1]
    families[stem].add(word)
    

In [32]:
print(families['book'])
print(families['thwart'])

{'books', 'bookers', 'booker', 'bookings', 'book', "book's", 'booked', 'booking'}
{'thwart', 'thwarting', 'thwarted'}


In [33]:
# hidden tests happen here

### Detect errors (6pt)

Use the dictionary of stems to morphological families to detect at least two instances of each of the following two types of errors (and of course you can't take the examples given here!). Use your own judgment of morphological boundaries to determine if analyses are correct or not.

* a case of undersegmentation, where no morphological boundary is given where one should be there (e.g., _expediting_ not being split into _expedit_ and _-ing_)
* a case of oversegmentation, where a morphological boundary is given where none should be there (e.g., _shoulder_ being erroneously split into _should_ and _-er_)

Next, discuss (1) how these errors (likely) have arisen and (2) how they could have been prevented.

In [34]:
print(len(families['expedit']))
print(len(families['should']))

0
5


In [35]:
for key in families.keys():
    if len(families[key]) == 0:
        print(key)

expedit


### Undersegmentation

In [36]:
parse_word('writing', derived)

[('suffix', 'ing'), ('suffix', ''), 'writ']

In [37]:
parse_word('handwriting', derived)

['handwriting']

### Oversegmentation

In [38]:
parse_word('power', derived)

[('suffix', 'er'), ('suffix', ''), 'pow']

In [39]:
print(families['pow'])

{'powers', "power's", 'power', 'pow'}


In the case of undersegmentation, `writing` can be parsed into `writ` and `ing`. But after `hand` is added at the beginning, it can no longer be parsed anymore. I think the reason for this to happen is both `hand` and `writing` have their own meaning, so the function may be confused about what to do when they are combined into one word. In order to avoid this to happen, we can change the input to a verb or a noun that can not be seprerated as two different words.

In the case of oversegmentation, `power` is parsed into `pow` and `er`. After I checked `pow` in the dictionary, I think the reason for this to happen is the function is treating `pow` as the stem, and `er` as the suffix. It can not recognize `power` as an individual word. One possible way to improve this is to use the NLTK Classifier to read the same word for a few times until it can recognize the word correctly. So that the function can treat `pow` and `power` differently by their meanings.

## Part 3: Applying the model to a novel language

The great thing about this algorithm is that you can apply it to a language you know nothing about and try to see if it comes up with meaningful morphemes.

Each of you have a language assigned to you (see http://www.cs.toronto.edu/~barend/lin340_final_assignment.txt). The filename given there under the <filename\> column contains a word list of your language. The URL also provide you with the Wikipedia page of that language so you can find out more about your language.

If you assign the filename of your language as a string to `lg` below, the code will read in the word list and runs the `learn_morphology` function on the ite

In [40]:
fn = 'eus.txt'
words = [l.strip('\n') for l in open(fn).readlines()]
unmodeled = set(words)
derived = learn_morphology(unmodeled, 5, 500, 20)

best pair: ('suffix', '') ('suffix', 'a') 	 stems 891 n unmodeled 12671 n analyzed 0
best pair: ('suffix', '') ('suffix', 'ak') 	 stems 629 n unmodeled 11780 n analyzed 1782
best pair: ('suffix', '') ('suffix', 'ko') 	 stems 455 n unmodeled 11151 n analyzed 2619
best pair: ('suffix', '') ('suffix', 'rik') 	 stems 353 n unmodeled 10696 n analyzed 3299
best pair: ('suffix', '') ('suffix', 'z') 	 stems 335 n unmodeled 10343 n analyzed 3787
best pair: ('suffix', '') ('suffix', 'k') 	 stems 230 n unmodeled 10008 n analyzed 4194
best pair: ('suffix', '') ('suffix', 'en') 	 stems 227 n unmodeled 9778 n analyzed 4573
best pair: ('suffix', '') ('suffix', 'aren') 	 stems 201 n unmodeled 9554 n analyzed 4812
best pair: ('suffix', 'n') ('suffix', 'ko') 	 stems 199 n unmodeled 9353 n analyzed 5011
best pair: ('suffix', '') ('suffix', 'ra') 	 stems 230 n unmodeled 9154 n analyzed 5386
best pair: ('suffix', '') ('suffix', 'an') 	 stems 199 n unmodeled 8924 n analyzed 5674
best pair: ('suffix', '') ('

In [41]:
print(unmodeled)

{'hemezortzi', 'xurgatu', 'handiago', 'hango', 'bainauzue', 'niregandik', 'guztiaz', 'goien', 'isil-isilik', 'bilatu', 'nahastera', 'garenontzat', 'gaituzte', 'azkenetan', 'bestearenagatik', 'epikureo', 'gordina', 'zapuztu', 'begiramenez', 'lotsatzen', 'direnik', 'palma-adarrak', 'laguntzaileoi', 'genbiltzan', 'epel', 'aukeratzen', 'trumoi', 'larrialdi', 'gaurdanik', 'erreinukoek', 'ostalariari', 'galtzea', 'nabariak', 'ditiat', 'aitari', 'didate', 'baliotsuzko', 'oholtxo', 'korrika', 'prestakizunak', 'bizitze', 'kondenatutzat', 'eskabide', 'itxurazaleek', 'odol', 'oroitzapena', 'mesedegarri', 'magalean', 'arindu', 'mahaiburuak', 'errementariak', 'sinesmenetik', 'nerau', 'baditugu', 'murgiltze', 'iraunkor', 'geldi-geldi', 'maila', 'bekie', 'besarkada', 'ematerik', 'atsedenik', 'itsukeriak', 'edonor', 'nenbilela', 'lurrekoaz', 'baitzerion', 'batengana', 'gozotasunez', 'hartzarenak', 'ur-boladak', 'abiatzera', 'ongiarekin', 'fariseuok', 'ahalegin', 'senideoi', 'badakizuelako', 'piztuerak

### Discuss the affixes and derivations the model finds (6pt)

Using the word list and the derivation dictionary, try to explore the affixes you extracted for your language. Use techniques such as the extraction of morphological families, as well as the on-screen print-out of the `learn_morphology` function. Use the Wikipedia page to see if you can determine what the affixes are (are they case markers, or tense-aspect-modality markers, or gender, or evidentiality, or ...).  Leave the code you use for the exploration up so I can see it. Use the text cell at the very bottom to describe your findings. Include discussion of the affixes you failed to find in your description.

In [42]:
words

['a',
 'abantailarik',
 'abar',
 'abegi',
 'aberasbide',
 'aberasgarriago',
 'aberastasun',
 'aberastasuna',
 'aberastasunak',
 'aberastasunarekin',
 'aberastasunari',
 'aberastasunen',
 'aberasten',
 'aberastu',
 'aberastuak',
 'aberats',
 'aberatsa',
 'aberatsak',
 'aberatsaren',
 'aberatsei',
 'aberatsez',
 'aberatsok',
 'abere',
 'abere-aska',
 'abere-odola',
 'aberea',
 'abereak',
 'aberearena',
 'abereek',
 'abereen',
 'abererik',
 'aberriaren',
 'abesten',
 'abestu',
 'abestuz',
 'abia',
 'abiatu',
 'abiatuko',
 'abiaturik',
 'abiatzeko',
 'abiatzera',
 'abisua',
 'abokatu',
 'adabaki',
 'adabakia',
 'adabakiak',
 'adabakirik',
 'adar',
 'adar-hostoak',
 'adarrak',
 'adarrek',
 'adarretan',
 'adarrok',
 'adaxkak',
 'adi-adi',
 'adibidea',
 'adibideak',
 'adibidez',
 'adieraz',
 'adierazgarri',
 'adierazi',
 'adierazia',
 'adieraziko',
 'adierazirik',
 'adierazitako',
 'adieraziz',
 'adierazpen',
 'adierazpena',
 'adieraztea',
 'adieraztean',
 'adierazteari',
 'adierazteko',
 'ad

In [43]:
parse_word('aberastuak', derived)

[('suffix', 'ak'), ('suffix', ''), 'aberastu']

In [44]:
print(families['aberastu'])

set()


In [48]:
parse_word('adierazteko', derived)

[('suffix', 'ko'), ('suffix', ''), 'adierazte']

In [46]:
parse_word('aberatsaren', derived)

[('suffix', 'aren'), ('suffix', ''), 'aberats']

In [47]:
parse_word('adierazteari', derived)

[('suffix', 'ari'), ('suffix', ''), 'adierazte']

The language I am assigned to analyze is Basque. After collecting from the txt file, the word I choose from  `words` is `'aberastuak'`. When I parse it into affixes and stem, the result shows it contains an empty suffix `''` and a suffix `'ak'`. And the stem here is `'aberastu'`. 

When I check from the google translate, both the word `'aberastuak'` and the stem `'aberastu'` are translated to be the verb `'enriched'` in English. So my first observation here is the suffix `-ak` doesn't change the tense of the verb. From Wikipedia, we know there are three arguments that are relate to verbs of Basque, which are subject, direct object and indirect object. And the `-ak` only show up in absolutive and ergative cases, corresponding to the finding we have for the verb.

When I translate from `'enrich'` as a possible stem from English, the word in Basque becomes `'aberastu'`, with the origal Basque word `'aberastuak'` to be one of the derivations (Google Translate). As a result, we can use machine translation bidirectionally between English and Basque, with a certain level of limitation by further research to find out.

Similarly, I try `-ko` within the word `'adierazteko'`. According to Wikipedia, the suffix here functions as the case forms to the word adjective. For the suffix `-aren` within the word `'aberatsaren'`, it is one of the possesive genitive in the language. For the suffix `-ari` within the word `'adierazteari'`, it is the dative suffix case in the language.

In conclusion, I found most of the suffixes from the `derived` dictionary are consistent with the `parsed_word` and the explanation from the Wikipedia page of Basque grammar.

